In [18]:
import os
import datetime as dt
import pandas as pd
import shutil

# Get today's date
date_today = dt.datetime.today().strftime('%Y%m%d')

# Define the path to the Incoming_Files directory for today's date
incoming_path = f'Incoming_Files/{date_today}'
success_path = f'Success_Files/{date_today}'
rejected_path = f'Rejected_Files/{date_today}'

# Check if the directory exists
if os.path.isdir(incoming_path):
    # List all files in the directory
    files_in_directory=os.listdir(incoming_path)
    
    # Loop over each file in the directory
    for order in files_in_directory:
        header = False
        reject_reason = ''
        rejected_count=0
        success_count=0
        
        # Construct the full file path
        path_of_order = os.path.join(incoming_path, order)
        
        df_order = pd.read_csv(path_of_order)
        print(df_order)

        df_product_master=pd.read_csv(f'Product_Master/product_master.csv')
        print(df_product_master)

        product_id_master_list=df_product_master['product_id'].values
        print(product_id_master_list)

        #Validation1
        for value  in df_order['product_id']:
            if value not in product_id_master_list:
                pid_reject_reason = f"Invalid product_id {value}"
                reject_reason = reject_reason + pid_reject_reason + ';'
                print(reject_reason)     

        #Validation2
        combined_df = pd.merge(left = df_order, right = df_product_master, how = 'inner', on = 'product_id')
        print(combined_df)

        for i in range(len(combined_df)):
            sales_value = combined_df.iloc[i]['sales']
            quantity_value = combined_df.iloc[i]['quantity']
            price_value = combined_df.iloc[i]['price']

            if sales_value != (quantity_value * price_value):
                sales_reject_reason = f"Invalid sales for product_id {combined_df.iloc[i]['product_id']}"
                reject_reason = reject_reason + sales_reject_reason + ';'
                print(reject_reason)

        #Validation3
        for i in range(len(df_order)):
            order_date_val = df_order.iloc[i]['order_date']
            if order_date_val > date_today:
                date_reject_reason = f"Date is in the future {df_order['order_date']}"
                reject_reason = reject_reason + date_reject_reason + ';'
                print(reject_reason)

        #Validation4
        if df_order.isna().any().any():
            empty_reject_reason = f"Empty value"
            reject_reason = reject_reason + empty_reject_reason + ';'
            print(reject_reason)

        #Validation5
        for i in range(len(df_order)):
            if df_order.iloc[i]['city'] not in ('Mumbai','Bangalore'):
                city_reject_reason = f"City {df_order.iloc[i]['city']} not in Mumbai and Bangalore"
                reject_reason = reject_reason + city_reject_reason + ';'
                print(reject_reason)

        if reject_reason=='':
            success_count +=1
            shutil.copy(path_of_order, success_path)
        else:
            row_str=''
            if not os.path.exists(f'{rejected_path}'):
                os.makedirs(f'{rejected_path}', exist_ok=True)
            shutil.copy(f'{incoming_path}/{order}', f'{rejected_path}/{order}')
            rejected_count += 1

            

        
else:
    print(f"Directory not found: {incoming_path}")


   order_id  order_date  product_id  quantity  sales       city
0         1  2023-01-01         100         1  50000  Bangalore
1         2  2023-02-01         200         2   9000     Mumbai
2         3  2023-01-03         200         2   4500     Mumbai
3         4  2023-01-04         800         3  30000     Mysore
   product_id product_name  price     category
0         100       iphone  50000       mobile
1         200        chair   4500    furniture
2         300        table  10000    furniture
3         400           tv  25000  electronics
4         500       fridge  10000  electronics
[100 200 300 400 500]
Invalid product_id 800;
   order_id  order_date  product_id  quantity  sales       city product_name  \
0         1  2023-01-01         100         1  50000  Bangalore       iphone   
1         2  2023-02-01         200         2   9000     Mumbai        chair   
2         3  2023-01-03         200         2   4500     Mumbai        chair   

   price   category  
0  50000 